<a href="https://colab.research.google.com/github/mohitha-llm-scaleai/BigQuery-AI/blob/main/BigQueryAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = "genai-poc-424806"   # your project
LOCATION = "us"
print("OK:", PROJECT_ID, LOCATION)


OK: genai-poc-424806 us


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!gcloud config set project genai-poc-424806
!gcloud auth list


Updated property [core/project].
    Credentialed Accounts
ACTIVE  ACCOUNT
*       mohitha@llmatscale.ai

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [ ]:
!bq --project_id=genai-poc-424806 mk --connection --location=US \
  --connection_type=CLOUD_RESOURCE vertex_conn


BigQuery error in mk operation: Already Exists: Connection
projects/398219119144/locations/us/connections/vertex_conn


In [ ]:
!bq --project_id=genai-poc-424806 ls --connection --location=US

             name                 friendlyName      description    Last modified         type        hasCredential                                            properties                                            
 ----------------------------- ------------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  398219119144.us.vertex_conn   Vertex Connection                 02 Sep 04:48:27   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-398219119144-imuz@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  


In [ ]:
!bq show --format=prettyjson genai-poc-424806:google_patents_research | grep location


In [ ]:
!bq ls --project_id=genai-poc-424806


      datasetId      
 ------------------- 
  Adult_data         
  Fusable            
  Gallo_demo         
  SerranoAdvisorsBQ  
  TEST               
  juvenile_tables    
  patent_demo        


In [ ]:
!bq ls genai-poc-424806:patent_demo


       tableId        Type    Labels   Time Partitioning   Clustered Fields  
 ------------------- ------- -------- ------------------- ------------------ 
  patent_docs         TABLE                                                  
  patent_embeddings   TABLE                                                  


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE MODEL `patent_demo.embedding_model`
  REMOTE WITH CONNECTION `us.vertex_conn`
  OPTIONS (ENDPOINT = 'text-embedding-004');  -- or 'gemini-embedding-001'


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project genai-poc-424806
CREATE TABLE IF NOT EXISTS `patent_demo.patent_embeddings` (
  publication_number STRING,
  title STRING,
  abstract STRING,
  text_embedding ARRAY<FLOAT64>
);


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project genai-poc-424806
SELECT column_name, data_type
FROM `genai-poc-424806.patent_demo`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'patent_docs'
ORDER BY ordinal_position;


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,publication_number,STRING
1,title,STRING
2,abstract,STRING
3,filing_date,INT64
4,assignee,ARRAY<STRING>
5,cpc,"ARRAY<STRUCT<code STRING, inventive BOOL, firs..."


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE TABLE `patent_demo.patent_embeddings` AS
WITH src AS (
  SELECT
    d.publication_number,
    CAST(d.title    AS STRING) AS title_text,
    CAST(d.abstract AS STRING) AS abstract_text
  FROM `patent_demo.patent_docs` d
),
gen AS (
  SELECT
    publication_number,
    ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (
      SELECT
        publication_number,
        CONCAT(COALESCE(title_text, ''), ' ', COALESCE(abstract_text, '')) AS content
      FROM src
      WHERE (title_text IS NOT NULL OR abstract_text IS NOT NULL)
      ORDER BY publication_number

    )
  )
)
SELECT
  s.publication_number,
  s.title_text   AS title,
  s.abstract_text AS abstract,
  g.text_embedding
FROM gen g
JOIN src s USING (publication_number);


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project genai-poc-424806
SELECT COUNT(*) AS rows_with_embeddings FROM `patent_demo.patent_embeddings`;


Query is running:   0%|          |

Downloading:   0%|          |

,rows_with_embeddings
0,19989


In [ ]:
import os
PROJECT_ID = "genai-poc-424806"   # replace with your actual project
os.environ["PROJECT_ID"] = PROJECT_ID


In [ ]:
%%bigquery --project genai-poc-424806
CREATE VECTOR INDEX `patent_demo.idx_patent_embeddings`
ON `patent_demo.patent_embeddings`(text_embedding)
STORING(publication_number, title, abstract)               -- optional but recommended
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{"num_lists": 500}'                       -- tune as needed (≤ 5000)
);


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project genai-poc-424806
-- High-level: one row per vector index in the dataset
SELECT *
FROM `patent_demo`.INFORMATION_SCHEMA.VECTOR_INDEXES;


Query is running:   0%|          |

Downloading:   0%|          |

,index_catalog,index_schema,table_name,index_name,index_status,creation_time,last_modification_time,last_refresh_time,disable_time,disable_reason,ddl,coverage_percentage,unindexed_row_count,total_logical_bytes,total_storage_bytes
0,genai-poc-424806,patent_demo,patent_embeddings,patent_demo.idx_patent_embeddings,ACTIVE,2025-09-04 15:59:57.506000+00:00,2025-09-04 15:59:57.506000+00:00,NaT,NaT,None,CREATE VECTOR INDEX `patent_demo.idx_patent_em...,0,19989,0,0


In [ ]:
%%bigquery --project genai-poc-424806
-- Options actually applied (index_type, distance_type, IVF params, etc.)
SELECT *
FROM `patent_demo`.INFORMATION_SCHEMA.VECTOR_INDEX_OPTIONS;

Query is running:   0%|          |

Downloading:   0%|          |

,index_catalog,index_schema,table_name,index_name,option_name,option_type,option_value
0,genai-poc-424806,patent_demo,patent_embeddings,patent_demo.idx_patent_embeddings,index_type,STRING,IVF
1,genai-poc-424806,patent_demo,patent_embeddings,patent_demo.idx_patent_embeddings,distance_type,STRING,COSINE
2,genai-poc-424806,patent_demo,patent_embeddings,patent_demo.idx_patent_embeddings,ivf_options,STRING,"{""num_lists"": 500}"


In [ ]:
%%bigquery --project genai-poc-424806
DECLARE user_query STRING DEFAULT 'battery thermal runaway prevention in EVs';

-- Build a "query table" with the same column name as the base embedding column
WITH q AS (
  SELECT ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (SELECT user_query AS content)
  )
)
SELECT
  base.publication_number,
  base.title,
  base.abstract,
  distance
FROM VECTOR_SEARCH(
  TABLE `patent_demo.patent_embeddings`,   -- base table
  'text_embedding',                         -- embedding column in base
  TABLE q,                                  -- one-row query table with matching column name
  top_k => 8,
  distance_type => 'COSINE'
);


Query is running:   0%|          |

Downloading:   0%|          |

,publication_number,title,abstract,distance
0,US-12155049-B2,Systems and methods for battery thermal manage...,"An electrified vehicle includes a chassis, an ...",0.349070
1,CN-115379963-A,Battery pack selection for pre-charging of DC bus,The present disclosure provides a system and m...,0.368115
2,US-2023253781-A1,Protection circuit for battery management system,Systems and methods are provided for a battery...,0.376454
3,US-11573271-B1,Battery fault detection,"A control system, responsive to receiving valu...",0.383261
4,US-2024345175-A1,Real-time battery fault detection and state-of...,Battery management systems and methods can pro...,0.392394
5,US-11529875-B1,Electrified vehicle configured to disconnect b...,This disclosure relates to an electrified vehi...,0.394930
6,CA-3081042-A1,"Systeme, dispositif et procede de stockage d`e...",A distributed battery module thermal managemen...,0.423772
7,TW-I838109-B,Battery system for electric vehicles and contr...,本發明提出一種用於電動車輛的電池系統。電動車輛包括馬達以及整車控制器。所述電池系統包括第一電...,0.425908


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE MODEL `patent_demo.gemini_text`
REMOTE WITH CONNECTION `us.vertex_conn`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project genai-poc-424806
DECLARE user_query STRING DEFAULT 'Summarize top innovations that reduce EV battery fire risk.';

WITH q AS (
  SELECT ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (SELECT user_query AS content)
  )
),
hits AS (
  SELECT
    base.publication_number,
    base.title,
    SUBSTR(base.abstract, 1, 1200) AS abstract,
    distance
  FROM VECTOR_SEARCH(
    TABLE `patent_demo.patent_embeddings`,
    'text_embedding',
    TABLE q,
    top_k => 8,
    distance_type => 'COSINE'
  )
  ORDER BY distance
),
context AS (
  SELECT STRING_AGG(
    CONCAT('PUB: ', publication_number, '\nTITLE: ', title, '\nABSTRACT: ', abstract),
    '\n\n---\n\n'
    ORDER BY distance
  ) AS ctx
  FROM hits
)
SELECT *
FROM ML.GENERATE_TEXT(
  MODEL `patent_demo.gemini_text`,
  (SELECT CONCAT(
      'You are a precise patent analyst. Use ONLY the CONTEXT. ',
      'If info is missing, say so. Provide bullet points and cite PUB IDs used.\n\n',
      'QUESTION: ', user_query, '\n\n',
      'CONTEXT:\n', ctx
  ) AS prompt FROM context),
  STRUCT(0.2 AS temperature, 900 AS max_output_tokens)
);


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_result,ml_generate_text_status,prompt
0,"{""candidates"":[{""avg_logprobs"":-0.124375109850...",,You are a precise patent analyst. Use ONLY the...


In [ ]:
!bq --project_id=genai-poc-424806 ls -m patent_demo


        Id          Model Type   Labels    Creation Time   
 ----------------- ------------ -------- ----------------- 
  embedding_model                         04 Sep 10:05:54  
  gemini_text                             04 Sep 16:16:16  


In [ ]:
# === Minimal RAG UI for BigQuery + Vertex AI (Colab) ===
# Type a query -> get answer (optionally show sources)

import os
from google.cloud import bigquery
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd

# ------- Configure your project / dataset / models / table -------
PROJECT_ID   = "genai-poc-424806"
DATASET      = "patent_demo"
EMB_MODEL    = f"{DATASET}.embedding_model"   # text-embedding-004
LLM_MODEL    = f"{DATASET}.gemini_text"       # gemini-2.0-flash
EMB_TABLE    = f"{DATASET}.patent_embeddings" # has column text_embedding ARRAY<FLOAT64>

# UI defaults (you can tweak live)
DEFAULT_QUERY      = "battery thermal runaway prevention in EVs"
TOP_K_DEFAULT      = 5          # 3–8 is a good range
MAX_CTX_CHARS      = 1200       # keep abstracts compact to avoid token limits
TEMPERATURE        = 0.2        # LITERAL gets embedded into SQL (TVF requires literals)
MAX_OUTPUT_TOKENS  = 900        # LITERAL
SHOW_SOURCES       = True       # set False to hide the sources table

# -----------------------------------------------------------------
os.environ["GCLOUD_PROJECT"] = PROJECT_ID
bq = bigquery.Client(project=PROJECT_ID)

def run_rag(user_query: str,
            top_k: int = TOP_K_DEFAULT,
            max_ctx_chars: int = MAX_CTX_CHARS,
            temperature: float = TEMPERATURE,
            max_output_tokens: int = MAX_OUTPUT_TOKENS):
    """
    Runs retrieval -> generation inside BigQuery.
    Notes:
      - user_query is passed as a parameter
      - TVF settings (temperature, max_output_tokens) MUST be literal constants, so we embed them.
    """
    # --- Main query: retrieve -> build context -> generate ---
    sql_answer = f"""
    DECLARE user_query STRING DEFAULT @user_query;

    WITH q AS (
      SELECT ml_generate_embedding_result AS text_embedding
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{EMB_MODEL}`,
        (SELECT user_query AS content)
      )
    ),
    hits AS (
      SELECT
        base.publication_number,
        base.title,
        SUBSTR(base.abstract, 1, {max_ctx_chars}) AS abstract,
        distance
      FROM VECTOR_SEARCH(
        TABLE `{EMB_TABLE}`,
        'text_embedding',
        TABLE q,
        top_k => {top_k},
        distance_type => 'COSINE'
      )
      ORDER BY distance
    ),
    context AS (
      SELECT STRING_AGG(
        CONCAT('PUB: ', publication_number, '\\nTITLE: ', title, '\\nABSTRACT: ', abstract),
        '\\n\\n---\\n\\n' ORDER BY distance
      ) AS ctx
      FROM hits
    ),
    gen AS (
      SELECT
        ml_generate_text_result AS gen_json
      FROM ML.GENERATE_TEXT(
        MODEL `{LLM_MODEL}`,
        (
          SELECT CONCAT(
            'You are a precise patent analyst. Use ONLY the CONTEXT. ',
            'If info is missing, say so. Provide bullet points and cite PUB IDs.\\n\\n',
            'QUESTION: ', user_query, '\\n\\n',
            'CONTEXT:\\n', ctx
          ) AS prompt
          FROM context
        ),
        STRUCT({temperature} AS temperature, {max_output_tokens} AS max_output_tokens)  -- must be literals
      )
    )
    SELECT JSON_VALUE(gen_json, '$.candidates[0].content.parts[0].text') AS answer
    FROM gen;
    """

    job = bq.query(
        sql_answer,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[bigquery.ScalarQueryParameter("user_query", "STRING", user_query)]
        ),
        location="US",
    )
    answer_df = job.result().to_dataframe()

    # --- Optional: fetch sources for display ---
    sources_df = pd.DataFrame()
    if SHOW_SOURCES:
        sql_sources = f"""
        DECLARE user_query STRING DEFAULT @user_query;

        WITH q AS (
          SELECT ml_generate_embedding_result AS text_embedding
          FROM ML.GENERATE_EMBEDDING(
            MODEL `{EMB_MODEL}`,
            (SELECT user_query AS content)
          )
        )
        SELECT
          base.publication_number,
          base.title,
          distance
        FROM VECTOR_SEARCH(
          TABLE `{EMB_TABLE}`,
          'text_embedding',
          TABLE q,
          top_k => {top_k},
          distance_type => 'COSINE'
        )
        ORDER BY distance;
        """
        sources_job = bq.query(
            sql_sources,
            job_config=bigquery.QueryJobConfig(
                query_parameters=[bigquery.ScalarQueryParameter("user_query", "STRING", user_query)]
            ),
            location="US",
        )
        sources_df = sources_job.result().to_dataframe()

    answer_text = (answer_df.iloc[0]["answer"] if not answer_df.empty else "")
    return answer_text, sources_df

# ----------------------------- UI -----------------------------
qbox = widgets.Textarea(
    value=DEFAULT_QUERY,
    placeholder="Type your question…",
    description="Query:",
    layout=widgets.Layout(width="100%", height="90px")
)
topk_slider = widgets.IntSlider(value=TOP_K_DEFAULT, min=1, max=10, step=1, description='top_k:')
run_btn = widgets.Button(description="Run", button_style="primary")
out = widgets.Output()

def on_click(_):
    out.clear_output()
    with out:
        print("Running…")
        try:
            answer, sources = run_rag(
                user_query=qbox.value,
                top_k=topk_slider.value,
                max_ctx_chars=MAX_CTX_CHARS,
                temperature=TEMPERATURE,
                max_output_tokens=MAX_OUTPUT_TOKENS
            )
            display(Markdown("### Answer"))
            display(Markdown(answer if isinstance(answer, str) and answer else "_(no text)_"))
            if SHOW_SOURCES:
                display(Markdown("### Sources (top-k)"))
                display(sources)
        except Exception as e:
            print("ERROR:", e)

run_btn.on_click(on_click)
display(qbox, topk_slider, run_btn, out)


Textarea(value='battery thermal runaway prevention in EVs', description='Query:', layout=Layout(height='90px',…

IntSlider(value=5, description='top_k:', max=10, min=1)

Button(button_style='primary', description='Run', style=ButtonStyle())

Output()